Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

data를 평범한 matrix로 변환?!
label을 one-hot encoding으로

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32) # image_size를 곱해서 하나로..
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32) # label을 one-hot으로
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

일단, 간단한 gradient descent로 logistic regression을 학습시켜보자.

tensorflow가 하는 일:
* 동작하는 것 보고 싶은 것 계산하는 것을 보여준다. input, variable, operation같은 것을 보여준다. computation graph에 node를 만들어준다. 아래 block에 모든게 포함되어 있다.

코드

* 이 그래프에서 연산을 할 수 있다. 그것도 너가 원하는 만큼 여러번(seesion.run()이라는 게 그것을 수행), ~해석 잘 모름. 이 런타임 연산은 아래 블록에 다 포함됨.

코드

자, 모든 데이터를 TensorFlow에 불러와보자. 그리고 우리 학습에 맞게 computation graph를 만들어보자.

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000 # train은 10000개만

graph = tf.Graph()
with graph.as_default(): # 그래프를 만듬

  # Input data. 입력
  # Load the training, validation and test data into constants that are
  # attached to the graph. / 모든 데이터 세트를 상수(constant)로 불러와서 graph에 적용(attach 붙)
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :]) # train은 10000개만
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 변수 (파라미터)
  # These are the parameters that we are going to be training. The weight / 얘네는 우리가 학습시킬 파라미터들이다.
  # matrix will be initialized using random values following a (truncated) / Weight matrix는 랜덤값으로 초기화 (truncated) normal distribution으로 초기화
  # normal distribution. The biases get initialized to zero. / bias는 0으로 초기화
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels])) # truncated normal distributino으로 초기화(input개수 : image*image, output개수 : 라벨 or 클래스 수)
  biases = tf.Variable(tf.zeros([num_labels])) # bias는 0으로
  
  # Training computation. / 굳이 번역 X (학습을 위한 계산 정도로 이해할 수 있을 듯)
  # We multiply the inputs with the weight matrix, and add biases. We compute / input과 weight matrix를 곱하고, bias를 더함.
  # the softmax and cross-entropy (it's one operation in TensorFlow, because / softmax와 cross-entropy를 계산할 것이다.(이 두 개가 tensorflow에는 하나의 function으로 구현되어있음)
  # it's very common, and it can be optimized). We take the average of this / training example로 cross-entropy의 평균을 구할 것이다.
  # cross-entropy across all training examples: that's our loss. / 그게 바로 loss를 의미한다.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer. / optimizer 최적화?!최적화기
  # We are going to find the minimum of this loss using gradient descent. / gradient descent로 loss최소값을 찾는다.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data. / 각 데이터에 대해 예측
  # These are not part of training, but merely here so that we can report / 학습하는 부분은 아니지만, 학습의 정확성을 보고하기 위해서
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

이 계산을 반복적으로 해보자:

In [5]:
num_steps = 801 # 반복 횟수

def accuracy(predictions, labels): # 라벨과 비교해서 예측값의 정확성 계산
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session: # 앞서 정의한 그래프를 수행(Session에 graph 설정)
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. / 이 연산은 이 그래프에서 한번만 한다. 파라미터들을 초기화 시켜줌.
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps): # 연산 반복
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays. / .run()으로 계산 진행. optimizer, loss와 training prediction을 돌려서 numpy array로 리턴해준다.
    _, l, predictions = session.run([optimizer, loss, train_prediction]) # 실행시켜서 3 개를 리턴 받아옴.(근데 optimizer는 버리는 듯)
    if (step % 100 == 0): # 100번마다 정확도 출력
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies. / .eval()은 run()같은 건데, 1개의 numpy array만 return한다. 그래프의 모든 dependency를 재계산?!(docs를 읽어볼 것).
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels)) #반복 끝나고 test label로 정확성 계산

Initialized
Loss at step 0: 17.844061
Training accuracy: 13.8%
Validation accuracy: 16.4%
Loss at step 100: 2.350036
Training accuracy: 72.2%
Validation accuracy: 72.3%
Loss at step 200: 1.888033
Training accuracy: 74.9%
Validation accuracy: 74.3%
Loss at step 300: 1.639844
Training accuracy: 76.1%
Validation accuracy: 74.9%
Loss at step 400: 1.471029
Training accuracy: 77.0%
Validation accuracy: 75.3%
Loss at step 500: 1.345355
Training accuracy: 77.9%
Validation accuracy: 75.7%
Loss at step 600: 1.246899
Training accuracy: 78.6%
Validation accuracy: 76.0%
Loss at step 700: 1.166813
Training accuracy: 78.9%
Validation accuracy: 76.1%
Loss at step 800: 1.099822
Training accuracy: 79.3%
Validation accuracy: 76.3%
Test accuracy: 82.7%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

이제부터 SGD로 해보자. 더 빠를 것이다.

그래프 자체는 비슷하나, 모든 training data를 constant node로 가지고 있지 않게 한다. 대신, `placeholder`라는 노드를 만들어, `session.run()`이 불릴 때마다 실제 데이터를 feed로 삼게 한다. (실시간성?!)

In [6]:
batch_size = 128 #작게..

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed / 입력.
  # at run time with a training minibatch. / training data는 minibatch를 실시간으로 먹이기 위해 placeholder를 쓴다.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized. / 랜덤화된 training data안에 offset을 만든다.
    # Note: we could use better randomization across epochs. / (시간?!) 랜덤을 더 잘 사용.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size) # offset은 minibatch를 만들기 위해서..
    # Generate a minibatch. / minibatch를 만든다.
    batch_data = train_dataset[offset:(offset + batch_size), :] 
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch. / 사전 준비 세션을 말하는 어디에서 미니배치를 먹는지
    # The key of the dictionary is the placeholder node of the graph to be fed, / 사전의 포인트는 먹히기 위해 그래프의 플홀 노드이다.
    # and the value is the numpy array to feed to it. / 그리고 그 값은 넘피 배열이다.
    # tensorflow의 docs에서 session.run()에 대한 부분을 보면, feed_dict은 다음과 같이 설명되어 있다.
    # 
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.393182
Minibatch accuracy: 9.4%
Validation accuracy: 15.4%
Minibatch loss at step 500: 1.476587
Minibatch accuracy: 76.6%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 1.563313
Minibatch accuracy: 76.6%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 1.233701
Minibatch accuracy: 74.2%
Validation accuracy: 78.0%
Minibatch loss at step 2000: 0.945946
Minibatch accuracy: 78.1%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 0.959727
Minibatch accuracy: 82.0%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 1.134969
Minibatch accuracy: 78.9%
Validation accuracy: 79.3%
Test accuracy: 85.8%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

logistic regression SGD을 ReLU(nn.relu())와 1024개의 hidden node를 이용해서 1-hidden neural network로 바꿔라. 이 모델은 validation/test accuracy를 향상시켜준다.

---

In [8]:
batch_size = 128
hidden_node = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed / 입력.
    # at run time with a training minibatch. / training data는 minibatch를 실시간으로 먹이기 위해 placeholder를 쓴다.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node]))
    biases1 = tf.Variable(tf.zeros([hidden_node]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_node, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    """
    def forward_prop(inp):
        hidden_layer1 = tf.nn.relu(tf.matmul(inp, weights1)+biases1)
        return tf.matmul(hidden_layer1, weights2) + biases2
    """
    # Training computation.
    """
    logits = forward_prop(tf_train_dataset)
    """
    hidden_layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1)+biases1)
    logits = tf.matmul(hidden_layer1, weights2) + biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1)+biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(layer1_valid, weights2) + biases2) # just softmax(forward_prop(tf_valid_dataset))
    
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1)+biases1)
    test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weights2) + biases2) # just softmax(forward_prop(tf_test_dataset))

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized. / 랜덤화된 training data안에 offset을 만든다.
    # Note: we could use better randomization across epochs. / (시간?!) 랜덤을 더 잘 사용.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size) # offset은 minibatch를 만들기 위해서..
    # Generate a minibatch. / minibatch를 만든다.
    batch_data = train_dataset[offset:(offset + batch_size), :] 
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch. / 사전 준비 세션을 말하는 어디에서 미니배치를 먹는지
    # The key of the dictionary is the placeholder node of the graph to be fed, / 사전의 포인트는 먹히기 위해 그래프의 플홀 노드이다.
    # and the value is the numpy array to feed to it. / 그리고 그 값은 넘피 배열이다.
    # tensorflow의 docs에서 session.run()에 대한 부분을 보면, feed_dict은 다음과 같이 설명되어 있다.
    # 
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 287.765808
Minibatch accuracy: 11.7%
Validation accuracy: 30.9%
Minibatch loss at step 500: 31.623407
Minibatch accuracy: 78.1%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 18.707457
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 1500: 5.738024
Minibatch accuracy: 78.9%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 8.182550
Minibatch accuracy: 82.0%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 4.700353
Minibatch accuracy: 84.4%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 3.584045
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%
Test accuracy: 88.3%
